## Train LSTM model for mortality risk prediction

## Imports

In [ ]:
import random
import sys
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

sys.path.append("..")
sys.path.append("../..")
from drift_detection.baseline_models.temporal.pytorch.optimizer import Optimizer
from drift_detection.baseline_models.temporal.pytorch.utils import *

## Load train/val/test inputs and labels

In [ ]:
DIR = "/mnt/nfs/project/delirium/drift_exp/_extract_v2"

X_train = np.load(os.path.join(DIR, "X_train.npy"))
X_val = np.load(os.path.join(DIR, "X_val.npy"))
X_test = np.load(os.path.join(DIR, "X_test.npy"))

y_train = np.load(os.path.join(DIR, "y_train.npy"))
y_val = np.load(os.path.join(DIR, "y_val.npy"))
y_test = np.load(os.path.join(DIR, "y_test.npy"))

print("Train set size:", X_train.shape[0])
print("Val set size:", X_val.shape[0])
print("Test set size:", X_test.shape[0])

print("Num features:", X_train.shape[2])
print(X_train.shape, X_val.shape, X_test.shape)

## Model and training configuration

In [ ]:
output_dim = 1
batch_size = 32
input_dim = X_train.shape[2]
timesteps = X_train.shape[1]
hidden_dim = 128
layer_dim = 2
dropout = 0.25
n_epochs = 100
learning_rate = 5e-3
weight_decay = 1e-6
last_timestep_only = False

device = get_device()

X_train_inputs = X_train
X_val_inputs = X_val
X_test_inputs = X_test

train_dataset = get_data(X_train_inputs, y_train)
train_loader = train_dataset.to_loader(batch_size, shuffle=True)

val_dataset = get_data(X_val_inputs, y_val)
val_loader = val_dataset.to_loader(batch_size)

In [ ]:
model_params = {
    "device": device,
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "layer_dim": layer_dim,
    "output_dim": output_dim,
    "dropout_prob": dropout,
    "last_timestep_only": last_timestep_only,
}

model = get_temporal_model("lstm", model_params).to(device)

## Training and validation

In [ ]:
loss_fn = nn.BCEWithLogitsLoss(reduction='none')
optimizer = optim.Adagrad(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay
)
activation = nn.Sigmoid()
opt = Optimizer(
    model=model, loss_fn=loss_fn, optimizer=optimizer, activation=activation
)
opt.train(
    train_loader,
    val_loader,
    batch_size=batch_size,
    n_epochs=n_epochs,
    n_features=input_dim,
    timesteps=timesteps,
)
opt.plot_losses()

## Testing, metrics

In [ ]:
test_dataset = get_data(X_test_inputs, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
y_test_labels, y_pred_values, y_pred_labels = opt.evaluate(
    test_loader, batch_size=1, n_features=input_dim, timesteps=timesteps
)
y_pred_values = y_pred_values[y_test_labels != -1]
y_pred_labels = y_pred_labels[y_test_labels != -1]
y_test_labels = y_test_labels[y_test_labels != -1]
pred_metrics = print_metrics_binary(y_test_labels, y_pred_values, y_pred_labels)
pred_metrics